In [1]:
%load_ext autoreload
%autoreload 2

In [68]:
from sqlalchemy import select
from pastiche import tables
from pastiche.database import SessionLocal
from datetime import date

In [57]:
with SessionLocal() as db:
    solutions = db.query(tables.JumbleGame.solution).all()
    print(solutions[0])

('ITWASACINCH',)


# OpenAI exploration

In [9]:
import openai
from pydantic import Field
from openai_function_call import OpenAISchema

In [45]:
# class JumbleClue(OpenAISchema):
#     """Class representing a sentence used as a clue for a Jumble game"""
#     clue_sentence: str = Field(..., description="Sentence with gaps to be filled.")
#     solution: str = Field(..., description="The words to use to fill the gaps in the clue-sentence. The solution can be made of multiple words and these words are not separated by a space.")
#     full_sentence: str = Field(..., description="The full sentence with all words.")
    
# class JumbleCluesCollection(OpenAISchema):
#     """List of jumble clues"""
#     clues: list[JumbleClue]
    
# prompt1 = """
# Consider the data below: '\n{data}'. 
#                 For each entry in the list, follow the following steps:
#                 1. identify where are the gaps in the `clue_sentence`.
#                 2. reconstruct the correct words using the `solution` by adding missing spaces
#                 3. use the reconstructed solution to create a `full_sentence` by filling the gaps in the `clue_sentence`
# """


In [59]:
class JumbleClue(OpenAISchema):
    """Class representing a sentence used as a clue for a Jumble game"""
    word: str = Field(..., description="A list of letters defining one or multiple words without any space")
    word_with_spaces: str | None = Field(..., description="list of words recreated from `word` with the missing spaces added")
    
class JumbleCluesCollection(OpenAISchema):
    """List of jumble clues"""
    clues: list[JumbleClue]
   
prompt2 = """
Consider the data below: '\n{data}'. For each word, reconstruct the correct words by adding missing spaces.
"""

In [64]:
def generate_jumble_clues(data: str) -> JumbleCluesCollection:
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        temperature=0.1,
        functions=[JumbleCluesCollection.openai_schema],
        function_call={"name": JumbleCluesCollection.openai_schema["name"]},
        messages=[
            {
                "role": "user",
                "content": f"Consider the data below: '\n{data}'. For each word, reconstruct the correct words by adding missing spaces. If you can't find a match please return None."
            },
        ],
        max_tokens=1000,
    )
    return JumbleCluesCollection.from_response(completion)

In [65]:
results = generate_jumble_clues(
    data=solutions[:10]
)

In [66]:
results

JumbleCluesCollection(clues=[JumbleClue(word='ITWASACINCH', word_with_spaces=None), JumbleClue(word='ALLSHOOKUP', word_with_spaces=None), JumbleClue(word='VANNA', word_with_spaces=None), JumbleClue(word='CAROL', word_with_spaces=None), JumbleClue(word='HOLETRUTH', word_with_spaces=None), JumbleClue(word='GRANDTIME', word_with_spaces=None), JumbleClue(word='FLOCKEDTOIT', word_with_spaces=None), JumbleClue(word='INMANYRESPECTS', word_with_spaces=None), JumbleClue(word='WEIGHINGONHIM', word_with_spaces=None), JumbleClue(word='SEATRANQUILITY', word_with_spaces=None)])